In [1]:
import os
from ollama import Client, ResponseError
import pandas as pd
import numpy as np
import time
import struct

In [2]:
HUMAN_TEXT_DS = 'human-written-text-dataset.csv'
AI_FOLDER = 'ai'
GPT = 'gpt-oss:120b'
GEMINI = 'gemini-3-flash-preview:cloud'
MISTRAL = 'mistral-large-3:675b-cloud'
NUM_MODELS = 3

In [3]:
df = pd.read_csv(HUMAN_TEXT_DS)
df

,index,url,title,text,origin
0,0,NaN,تفاصيل آخر اللحظات بحياة ماجدة.. وسبب تأجيل إع...,\nالفنانة المصرية، ماجدة الصباحي، عن عالمنا قب...,ANAD
1,1,NaN,قصة مرض أجبر محمود ياسين على الاعتزال والبقاء ...,\nجديدة من أوراق الفن العربي سقطت، برحيل الفنا...,ANAD
2,2,NaN,لماذا تآمرت وفاء عامر على شقيقتها آيتن في الفن؟,\nقوية تجمع الفنانة #وفاء_عامر بشقيقتها الصغرى...,ANAD
3,3,NaN,#مرايا.. على العقيق اجتمعنا,"\nنرى صورة ماضينا؟, سؤال يترتب على جوابه نزاع ...",ANAD
4,4,NaN,هذا ما قاله محمد رمضان عن حلا شيحة,\nأخرى تجددت الأقاويل التي تتناول علاقة الفنان...,ANAD
...,...,...,...,...,...
995,995,NaN,العال الإسرائيلية تشغل 14 رحلة أسبوعين بين تل ...,\nمطارات دبي اليوم الرحلة الافتتاحية لخطوط الع...,ANAD
996,996,NaN,الصين تعلق رحلات الأفراد إلى تايوان,\nالصين، اليوم الأربعاء، أنها ستعلق اعتبارا من...,ANAD
997,997,NaN,"""السياحة المستدامة"" على رأس أولويات الفندقة با...",\nعلى البيئة والاستمتاع بخيراتها، لا يتعارض مع...,ANAD
998,998,NaN,بدون جواز سفر أو حجر.. تذاكر طيران نفدت في 10 ...,\nشركة كانتاس للطيران الأسترالية، اليوم الجمعة...,ANAD


In [4]:
human_text = df['text']
human_text

0      \nالفنانة المصرية، ماجدة الصباحي، عن عالمنا قب...
1      \nجديدة من أوراق الفن العربي سقطت، برحيل الفنا...
2      \nقوية تجمع الفنانة #وفاء_عامر بشقيقتها الصغرى...
3      \nنرى صورة ماضينا؟, سؤال يترتب على جوابه نزاع ...
4      \nأخرى تجددت الأقاويل التي تتناول علاقة الفنان...
                             ...                        
995    \nمطارات دبي اليوم الرحلة الافتتاحية لخطوط الع...
996    \nالصين، اليوم الأربعاء، أنها ستعلق اعتبارا من...
997    \nعلى البيئة والاستمتاع بخيراتها، لا يتعارض مع...
998    \nشركة كانتاس للطيران الأسترالية، اليوم الجمعة...
999    \nقطاع "الطيران الخاص" بشكل خاص في الكويت والخ...
Name: text, Length: 1000, dtype: object

In [5]:
parts = np.array_split(human_text, NUM_MODELS)
print(f'Number of parts: {len(parts)}')
for i, part in enumerate(parts):
    part.index = pd.RangeIndex(start=0, stop=len(part), step=1)
    print(f'Size of part {i}: {len(part)}', '\n')
    print(part[:3], '\n')

Number of parts: 3
Size of part 0: 334 

0    \nالفنانة المصرية، ماجدة الصباحي، عن عالمنا قب...
1    \nجديدة من أوراق الفن العربي سقطت، برحيل الفنا...
2    \nقوية تجمع الفنانة #وفاء_عامر بشقيقتها الصغرى...
Name: text, dtype: object 

Size of part 1: 333 

0    \nالرغم من إحراز تقدم كبير في زيادة معدلات الت...
1    \nالمركز الوطني الأميركي للإحصاءات الصحية أن أ...
2    \nبيانات محدثة نشرت، الأحد، أن أكثر من نصف الم...
Name: text, dtype: object 

Size of part 2: 333 

0    \nفريق ميلان في الإنفراد بصدارة الدوري الإيطال...
1    \nهيرفي رينارد، المدير الفني للمنتخب السعودي، ...
2    \nياسر المسحل رئيس الاتحاد السعودي لكرة القدم ...
Name: text, dtype: object 



d:\Artificial Intelligence Master\Master 2\Final Project\StartUp\Dataset Building\.env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [6]:
client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

prompt = lambda input: f'''
تعريف النسق المربّع:

هو أسلوب يُستخدم لإضافة نصوص جديدة داخل المقالات أو النصوص الأصلية دون حذف أو تعديل أي كلمة منها،

وتكون الإضافات على شكل فقرات عددها من 1 إلى 5 توضع بين [أقواس مربعة]، موزّعة في بدايات ووسط ونهايات الفقرات،

بنسبة لا تتجاوز 25٪ من حجم النص الكلي،

وبنفس لغة النص الأصلي (عربية أو إنجليزية).

طبق النسق المربع على النص التالي و لتكن إجابتك هي النص الجديد فقط كي أنسخه مباشرة:
{input}
'''

In [7]:
def fetch_ai_edited_text(model, part, model_name:str, sleep_duration=1):
    # ensure all the necessary folder do exist
    if not os.path.isdir(AI_FOLDER):
       os.makedirs(AI_FOLDER, exist_ok=True)
    if not os.path.isdir(model_name):
       os.makedirs(AI_FOLDER + '/' + model_name, exist_ok=True)
    
    # the path to store ai edited texts
    PATH = AI_FOLDER + '/' + model_name + '/'

    # the path to the file that contains the index at which fetching is stopped last time
    index_path = PATH + 'stopped_at.bin'

    # if index file does not exist, create it and initialize the index to 0. else, retrieve the index
    index = 0
    if not os.path.exists(index_path):
       with open(index_path, 'wb') as file:
          file.write(struct.pack('i', 0))
    else:
       with open(index_path, 'rb') as file:
          bin_data = file.read(4)
          index = struct.unpack('i', bin_data)[0]
    
    # specify where to start
    start = 0 if index == 0 else index + 1

    for i in range(start, len(part)):
        messages = [
            {
              'role': 'user',
              'content': prompt(part[i]),
            },
        ]

        with open(f'{PATH}{model_name.lower()}_{i}.txt', 'w', encoding='utf-8') as file:
          try:
            for p in client.chat(model, messages=messages, stream=True):
              file.write(p['message']['content'])
            print(f'{PATH}{model_name.lower()}_{i}.txt has been created.')
          except ResponseError as e:
              print(e)
              break
          
        with open(index_path, 'wb') as file:
           file.write(struct.pack('i', i))
        
        time.sleep(sleep_duration)

In [8]:
fetch_ai_edited_text(GPT, parts[0], 'GPT')

In [9]:
fetch_ai_edited_text(GEMINI, parts[1], 'GEMINI', sleep_duration=1)

In [11]:
fetch_ai_edited_text(MISTRAL, parts[2], 'MISTRAL')

ai/MISTRAL/mistral_318.txt has been created.
ai/MISTRAL/mistral_319.txt has been created.
ai/MISTRAL/mistral_320.txt has been created.
ai/MISTRAL/mistral_321.txt has been created.
ai/MISTRAL/mistral_322.txt has been created.
ai/MISTRAL/mistral_323.txt has been created.
ai/MISTRAL/mistral_324.txt has been created.
ai/MISTRAL/mistral_325.txt has been created.
ai/MISTRAL/mistral_326.txt has been created.
ai/MISTRAL/mistral_327.txt has been created.
ai/MISTRAL/mistral_328.txt has been created.
ai/MISTRAL/mistral_329.txt has been created.
ai/MISTRAL/mistral_330.txt has been created.
ai/MISTRAL/mistral_331.txt has been created.
ai/MISTRAL/mistral_332.txt has been created.
